In [1]:
import sys
from pathlib import Path

In [3]:
src_path = Path('../src').resolve()

sys.path.append(str(src_path))

In [7]:
from sqlmodel import Session,select
from api.db.session import engine
from api.events.models import EventModel
from timescaledb.hyperfunctions import time_bucket
from pprint import pprint

In [10]:
with Session(engine) as session:
    query=select(EventModel).order_by(EventModel.id.asc()).limit(10)
    compiled_query = query.compile(compile_kwargs={"literal_binds":True})
    print(compiled_query)
    print("")
    print(str(query))

SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.id ASC
 LIMIT 10

SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.id ASC
 LIMIT :param_1


In [ ]:

from sqlalchemy import func
from datetime import datetime,timedelta,timezone

with Session(engine) as session:
    bucket = time_bucket('1 minute',EventModel.time)
    pages = ['/about']#,'/contact','/pages','/pricing']
    start = datetime.now(timezone.utc) - timedelta(hours=1)
    finish = datetime.now(timezone.utc) + timedelta(hours=1)
    query=(
        select(
            bucket,
            EventModel.page,
            func.count()
            )
            .where(
                EventModel.time > start,
                EventModel.time<=finish,
                EventModel.page.in_(pages)
            )
            .group_by(
                bucket,
                EventModel.page
           )
           .order_by(
               bucket,
               EventModel.page
           )
        )
    compiled_query = query.compile(compile_kwargs={"literal_binds":True})
    results= session.exec(query).fetchall()
    pprint(results)

[(datetime.datetime(2025, 7, 22, 5, 43, tzinfo=datetime.timezone.utc), '/about', 522),
 (datetime.datetime(2025, 7, 22, 5, 44, tzinfo=datetime.timezone.utc), '/about', 576),
 (datetime.datetime(2025, 7, 22, 5, 45, tzinfo=datetime.timezone.utc), '/about', 625),
 (datetime.datetime(2025, 7, 22, 5, 46, tzinfo=datetime.timezone.utc), '/about', 547),
 (datetime.datetime(2025, 7, 22, 5, 47, tzinfo=datetime.timezone.utc), '/about', 189)]
